# <span style="color:green">Formation South Green 2022</span> - Structural Variants Detection by using short and long reads 

# __DAY 2 : SNP calling__

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)

## __1. Preparing the working environment__ 

### First create a dedicated folder to work 

In [1]:
# go to work directory
cd /home/jovyan/work/
ls

MAPPING-ILL  SV_DATA               training_SV_teaching_old
MAPPING-ONT  training_SV_teaching


### We are going to use only one clones to check is all is ok before run the whole of samples! 

In [6]:
## declare variables
i=10
REF_DIR="/home/jovyan/work/SV_DATA/REF/"
REF="/home/jovyan/work/SV_DATA/REF/reference.fasta"
ONT="/home/jovyan/work/SV_DATA/LONG_READS/Clone$i.fastq.gz"
ILL_R1="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R1.fastq.gz"
ILL_R2="/home/jovyan/work/SV_DATA/SHORT_READS/Clone${i}_R2.fastq.gz"
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.bam"

In [4]:
# go to SR mapping results
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}

In [5]:
ls /home/jovyan/work/MAPPING-ILL/dirClone${i}

Clone10.bam        Clone10.mappedpaired.flagstats  Clone10.SORTED.bam.bai
Clone10.flagstats  Clone10.SORTED.bam


### Mark duplications

In [6]:
echo -e "\nMarkDuplicates in Clone$i";
gatk MarkDuplicates -I $BAM -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
samtools index Clone$i.SORTED.MD.bam;


MarkDuplicates in Clone10
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar MarkDuplicates -I /home/jovyan/work/MAPPING-ILL/dirClone10/Clone10.SORTED.bam -M duplicates.10.metrics -O Clone10.SORTED.MD.bam
09:12:47.126 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Jun 17 09:12:47 UTC 2022] MarkDuplicates --INPUT /home/jovyan/work/MAPPING-ILL/dirClone10/Clone10.SORTED.bam --OUTPUT Clone10.SORTED.MD.bam --METRICS_FILE duplicates.10.metrics --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBE

In [8]:
# check MD bam
ls -l

total 80100
-rw-r--r-- 1 jovyan users 40019082 Jun 17 09:06 Clone10.bam
-rw-r--r-- 1 jovyan users      381 Jun 17 09:06 Clone10.flagstats
-rw-r--r-- 1 jovyan users      380 Jun 17 09:06 Clone10.mappedpaired.flagstats
-rw-r--r-- 1 jovyan users 18215336 Jun 17 09:06 Clone10.SORTED.bam
-rw-r--r-- 1 jovyan users     2856 Jun 16 14:29 Clone10.SORTED.bam.bai
-rw-r--r-- 1 jovyan users 23758262 Jun 17 09:12 Clone10.SORTED.MD.bam
-rw-r--r-- 1 jovyan users     2856 Jun 17 09:12 Clone10.SORTED.MD.bam.bai
-rw-r--r-- 1 jovyan users     3466 Jun 17 09:12 duplicates.10.metrics


### Indexing reference with GATK

In [9]:
cd $REF_DIR
samtools faidx $REF
gatk CreateSequenceDictionary -R $REF

Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar CreateSequenceDictionary -R /home/jovyan/work/SV_DATA/REF/reference.fasta
INFO	2022-06-17 09:13:16	CreateSequenceDictionary	Output dictionary will be written in /home/jovyan/work/SV_DATA/REF/reference.dict
09:13:16.268 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Jun 17 09:13:16 UTC 2022] CreateSequenceDictionary --REFERENCE /home/jovyan/work/SV_DATA/REF/reference.fasta --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RA

: 3

### SNP Calling for a Clone

In [10]:
# go to bam repertory
cd /home/jovyan/work/MAPPING-ILL/dirClone${i}
# change BAM file for MD one
BAM="/home/jovyan/work/MAPPING-ILL/dirClone10/Clone$i.SORTED.MD.bam"
# lauch GATK HaplotypeCaller
echo -e "\nCalling Clone$i";
gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O Clone$i.g.vcf -R $REF -ERC GVCF;


Calling Clone10
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar HaplotypeCaller --native-pair-hmm-threads 4 -I Clone10.SORTED.MD.bam -O Clone10.g.vcf -R /home/jovyan/work/SV_DATA/REF/reference.fasta -ERC GVCF
09:13:42.117 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
09:13:42.452 INFO  HaplotypeCaller - ------------------------------------------------------------
09:13:42.453 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.2.6.1
09:13:42.453 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
09:13:42.458 INFO  Hapl

: 1

In [11]:
head Clone$i.g.vcf

head: cannot open 'Clone10.g.vcf' for reading: No such file or directory


: 1

### Error produced here is normal ! RG is important in mapping step  !! 

Yes, you have to relaunch mapping with all samples ! BUT ...

Don't worry, bam files with correct `-R "@RG\tID:Clone${i}\tSM:Clone${i}"` parameter in bwa-mem2 command are available for downloading.

In [13]:
cd /home/jovyan/work/
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" wget https://itrop.ird.fr/sv-training/BAM_ILL.tar.gz
tar zxvf BAM_ILL.tar.gz
BAM_ILL="/home/jovyan/work/BAM_ILL"
rm BAM_ILL.tar.gz
ls $BAM_ILL

BAM_ILL/
BAM_ILL/Clone10.SORTED.bam
BAM_ILL/Clone11.SORTED.bam
BAM_ILL/Clone12.SORTED.bam
BAM_ILL/Clone13.SORTED.bam
BAM_ILL/Clone14.SORTED.bam
BAM_ILL/Clone15.SORTED.bam
BAM_ILL/Clone16.SORTED.bam
BAM_ILL/Clone17.SORTED.bam
BAM_ILL/Clone18.SORTED.bam
BAM_ILL/Clone19.SORTED.bam
BAM_ILL/Clone1.SORTED.bam
BAM_ILL/Clone20.SORTED.bam
BAM_ILL/Clone2.SORTED.bam
BAM_ILL/Clone3.SORTED.bam
BAM_ILL/Clone4.SORTED.bam
BAM_ILL/Clone5.SORTED.bam
BAM_ILL/Clone6.SORTED.bam
BAM_ILL/Clone7.SORTED.bam
BAM_ILL/Clone8.SORTED.bam
BAM_ILL/Clone9.SORTED.bam


In [ ]:
#wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" wget https://itrop.ird.fr/sv-training/BAM_ONT.tar.gz
#tar zxvf BAM_ONT.tar.gz
#BAM_ONT="/home/jovyan/work/BAM_ONT"
#rm BAM_ONT.tar.gz
#ls $BAM_ONT

## Calling all samples on one raw VCF with correct BAM files

In [4]:
BAM_ILL="/home/jovyan/work/BAM_ILL"
ls $BAM_ILL

Clone10.SORTED.bam         Clone18.SORTED.MD.bam      Clone7.SORTED.bam
Clone10.SORTED.bam.bai     Clone18.SORTED.MD.bam.bai  Clone7.SORTED.bam.bai
Clone10.SORTED.MD.bam      Clone19.SORTED.bam         Clone7.SORTED.MD.bam
Clone10.SORTED.MD.bam.bai  Clone19.SORTED.bam.bai     Clone7.SORTED.MD.bam.bai
Clone11.SORTED.bam         Clone19.SORTED.MD.bam      Clone8.SORTED.bam
Clone11.SORTED.bam.bai     Clone19.SORTED.MD.bam.bai  Clone8.SORTED.bam.bai
Clone11.SORTED.MD.bam      Clone1.SORTED.bam          Clone8.SORTED.MD.bam
Clone11.SORTED.MD.bam.bai  Clone1.SORTED.bam.bai      Clone8.SORTED.MD.bam.bai
Clone12.SORTED.bam         Clone1.SORTED.MD.bam       Clone9.SORTED.bam
Clone12.SORTED.bam.bai     Clone1.SORTED.MD.bam.bai   Clone9.SORTED.bam.bai
Clone12.SORTED.MD.bam      Clone20.SORTED.bam         Clone9.SORTED.MD.bam
Clone12.SORTED.MD.bam.bai  Clone20.SORTED.bam.bai     Clone9.SORTED.MD.bam.bai
Clone13.SORTED.bam         Clone20.SORTED.MD.bam      duplicates.10.metrics
Clone13.SORTED.bam

In [5]:
# create a work directory to SNP calling results
mkdir -p /home/jovyan/work/VCF
VCF_DIR="/home/jovyan/work/VCF"

### Markduplicates

In [8]:
cd $BAM_ILL
for i in {1..20}
    do
        samtools index Clone$i.SORTED.bam;
        echo -e "\nMarkDuplicates in Clone$i";
        gatk MarkDuplicates -I Clone$i.SORTED.bam -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
        samtools index Clone$i.SORTED.MD.bam;
    done


MarkDuplicates in Clone1
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar MarkDuplicates -I Clone1.SORTED.bam -M duplicates.1.metrics -O Clone1.SORTED.MD.bam
09:33:18.208 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Jun 17 09:33:18 UTC 2022] MarkDuplicates --INPUT Clone1.SORTED.bam --OUTPUT Clone1.SORTED.MD.bam --METRICS_FILE duplicates.1.metrics --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --

In [ ]:
cd $BAM_ILL
for i in {1..20}
    do
        echo -e "\nMarkDuplicates in Clone$i";
        gatk MarkDuplicates -I Clone$i.SORTED.bam -M duplicates.$i.metrics -O Clone$i.SORTED.MD.bam;
        samtools index Clone$i.SORTED.MD.bam;
    done

In [ ]:
cd $BAM_ILL
for i in {1..20}
    do
        # lauch GATK HaplotypeCaller
        echo -e "\nCalling Clone$i";
        gatk --java-options "-Xmx4g" HaplotypeCaller --native-pair-hmm-threads 4 -I Clone$i.SORTED.MD.bam -O $VCF_DIR/Clone$i.g.vcf -R $REF -ERC GVCF
    done


Calling Clone1
Using GATK jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar HaplotypeCaller --native-pair-hmm-threads 4 -I Clone1.SORTED.MD.bam -O /home/jovyan/work/VCF/Clone1.g.vcf -R /home/jovyan/work/SV_DATA/REF/reference.fasta -ERC GVCF
11:31:40.773 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/conda/share/gatk4-4.2.6.1-1/gatk-package-4.2.6.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
11:31:40.925 INFO  HaplotypeCaller - ------------------------------------------------------------
11:31:40.925 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.2.6.1
11:31:40.925 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
11:3

### CombineGVCF

In [ ]:
# change of work directory
cd $VCF_DIR
# Loop to inflate the --variant option
OPTION=""
for i in {1..20}
do
    OPTION="${OPTION} --variant Clone${i}.g.vcf"
done
echo $OPTION
# GATK
gatk CombineGVCFs -R $REF $OPTION -O rawSNP.vcf

### Have a Look to it combineGVCF

In [ ]:
head -n 1000 rawSNP.vcf | tail

### Compute the Genotypes

In [ ]:
gatk --java-options "-Xmx4g" GenotypeGVCFs -R $REF -V rawSNP.vcf -O output.vcf

### Compute the SNP density along the chromosomes

In [ ]:
echo -e "Reference\t1000000\n" > genome.txt
bedtools genomecov -bga -split -i output.vcf -g $REF > density.csv

In [ ]:
head density.csv

### Count the number of variants with `bcftools stat`

In [ ]:
bcftools stats output.vcf > output.vcf.stats